According to MIT Climate Portal, the urban heat island effect is a phenomenon whereby cities experience higher air temperatures than the surrounding countryside. Scientists refer to areas afflicted by these higher temperatures as urban heat islands, and people living in these areas are particularly exposed to the effects of climate change. (https://climate.mit.edu/explainers/urban-heat-islands).

Moreover, certain social groups are particularly vulnerable to crises, including climate change, for example, **children**, persons with disabilities, ethnic minorities, **migrant workers**, **displaced persons**, **older people**, and other socially marginalized groups. (https://www.worldbank.org/en/topic/social-dimensions-of-climate-change#1).

If we focus our attention on migrants (and in particular refugees) and displaced people, the UNHCR (UN Refugee Agency) states that the vast majority of refugees (approximately 78 percent) do not live in camps but in cities, and are therefore more vulnerable to the extreme heatwaves caused by climate change. In general urban locations offer more opportunities to live autonomously and find employment, but they also pose major challenges as refugees are often forced to share accommodation or live in non-functional public buildings, collective centers, slums or other types of informal settlements with substandard living conditions. (https://action.unrefugees.org/refugee-facts/camps/)

THe UNHCR provides a database with the demographics of **forcibly displaced population** including internally dispaced people, refugees, asylum-seekers and other people in need of international protection that settled in urban locations in the countries of asylum. Since there is no universal definition of urban, UNCHR classifies an urban location as a settlement with more than 5,000 inhabitants.(2021, 2022 Data: https://www.unhcr.org/refugee-statistics/download/?url=0quh0N)

The data were then filtered to identify the developing countries among the asylum countries according to the UN historical classification of developing and developed countries in 2022 and 2021 through a VLOOKUP:
https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Funstats.un.org%2Funsd%2Fmethodology%2Fm49%2Fhistorical-classification-of-developed-and-developing-regions.xlsx&wdOrigin=BROWSELINK


Data were cleaned to have data for both 2021 and 2022 for all the developing countries.
Major **urban refugees settlements** were then identified through web scraping from wikipedia (https://en.wikipedia.org/wiki/Urban_refugee#References). Data were harmonised for Country name and ISO to have consistent data among the used database.

Coordinates of the cities in Developing countries with urban refugees settlements were found to then identify:
* Highest temperatures 2020 - 2023 to identify temperature variations trend,
* Days with temperatures >= 35° Celsius (that, according to WHO, may cause heat-related diseases https://www.who.int/news-room/fact-sheets/detail/climate-change-heat-and-health).

According to WHO, infants or people who are over 60 years of age or that have chronic health conditions are those most affected by **heat-related diseases**, therefore the focus will be on developing countries hosting urban forcibly displaced population where babies < 4 years old and elderly > 60 years old are at least => 50 (Data filtered through Pivot Table)

In [1]:
#Check if we have access to the .csv

cat Cities.csv

Yerevan,Armenia
Baku,Azerbaijan
Ouagadougou,Burkina+Faso
Bujumbura,Burundi
Brazzaville,Congo
Abidjan,Cote+d'Ivoire
Kinshasa,Congo
Alexandria,Egypt
Cairo,Egypt
Giza,Egypt
Addis+Ababa,Ethiopia
Accra,Ghana
Delhi,India
Jakarta,Indonesia
Babylon,Iraq
Baghdad,Iraq
Bangkok,Thailand
Ankara,Turkey
Kampala,Uganda
Damascus,Syria
Dakar,Senegal
Tripoli,Libya
Rabat,Morocco
Mexico+City,Mexico
Kassala,Sudan
Juba,South+Sudan
Istanbul,Turkey
Islamabad,Pakistan
Hargeisa,Somalia
Harare,Zimbabwe
Kabul,Afghanistan
Niamey,Niger
Nairobi,Kenya
Maputo,Mozambique
Lagos,Nigeria
Kuala+Lumpur,Malaysia
Kigali,Rwanda
Khartoum,Sudan
Caracas,Venezuela
Aden,Yemen
'Amran,Yemen
Sana'a,Yemen
Mukalla,Yemen


In [8]:
#Test the API

curl "https://nominatim.openstreetmap.org/search?q=Accra,Ghana&format=json" 

[{"place_id":289735786,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"relation","osm_id":12803764,"lat":"5.5571096","lon":"-0.2012376","class":"boundary","type":"administrative","place_rank":16,"importance":0.5964183313392782,"addresstype":"city","name":"Accra","display_name":"Accra, Korle-Klottey Municipal District, Greater Accra Region, Ghana","boundingbox":["5.5171908","5.6673682","-0.2841318","-0.0548565"]}]


In [36]:
#fix the format

curl -s "https://nominatim.openstreetmap.org/search?q=Mukalla,Yemen&format=json"  | jq

[
  {
    "place_id": 69011860,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "node",
    "osm_id": 4298024474,
    "lat": "14.5424624",
    "lon": "49.1231345",
    "class": "place",
    "type": "city",
    "place_rank": 16,
    "importance": 0.4444200427268794,
    "addresstype": "city",
    "name": "المكلا",
    "display_name": "المكلا, مديرية مدينة المكلا, محافظة حضرموت, اليمن",
    "boundingbox": [
      "14.3824624",
      "14.7024624",
      "48.9631345",
      "49.2831345"
    ]
  },
  {
    "place_id": 69244949,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "way",
    "osm_id": 96509538,
    "lat": "14.54288285",
    "lon": "49.12486412931612",
    "class": "landuse",
    "type": "residential",
    "place_rank": 22,
    "importance": 0.2000099999999999,
    "addresstype": "residential",
    "name": "المكلا",
    "display_name": "المكلا, حي غار الأحمر, المكلا, مدي

In [34]:
#API with the Cities.csv file

echo "lat, long, city, addresstype" > Cities_Coordinates.csv
while read city
do    
    result=$(curl -s "https://nominatim.openstreetmap.org/search?q=${city}&format=json" | jq '.[] | [.lat, .lon, .name, .addresstype] | @csv')
    result2=$(echo "${result//\"/}")
    echo "${result2//\\/}"
done < Cities.csv >> Cities_Coordinates.csv

The Cities_Coordinates.csv was then clean by cities and municipalities, the arabic names were converted in English by ChatGPT for having consistent data for the final analysis.

I then used the API on https://open-meteo.com/en/docs/historical-weather-api to extract the **Maximum Temperature (2 m)** which is equal to the maximum temperature recorded at a height of 2 meters above the ground surface for all the identified cities from 01-01-2020 to 31-12-2023. 

In [3]:
#I tried the API with Addis_Ababa:
curl -s "https://archive-api.open-meteo.com/v1/archive?latitude=9.0358287&longitude=38.7524127&start_date=2020-01-01&end_date=2023-12-31&daily=temperature_2m_max,precipitation_sum" | jq

{
  "latitude": 9.033392,
  "longitude": 38.771626,
  "generationtime_ms": 0.6020069122314453,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2472,
  "daily_units": {
    "time": "iso8601",
    "temperature_2m_max": "°C",
    "precipitation_sum": "mm"
  },
  "daily": {
    "time": [
      "2020-01-01",
      "2020-01-02",
      "2020-01-03",
      "2020-01-04",
      "2020-01-05",
      "2020-01-06",
      "2020-01-07",
      "2020-01-08",
      "2020-01-09",
      "2020-01-10",
      "2020-01-11",
      "2020-01-12",
      "2020-01-13",
      "2020-01-14",
      "2020-01-15",
      "2020-01-16",
      "2020-01-17",
      "2020-01-18",
      "2020-01-19",
      "2020-01-20",
      "2020-01-21",
      "2020-01-22",
      "2020-01-23",
      "2020-01-24",
      "2020-01-25",
      "2020-01-26",
      "2020-01-27",
      "2020-01-28",
      "2020-01-29",
      "2020-01-30",
      "2020-01-31",
      "2020-02-01",
      "2020-02-02",
      

I then applied the API for all the selected cities. The code was extremely heavy and took about 20 minutes to run, and I found out that the same .json file could be downloaded directly from this link https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_max,precipitation_sum&location_mode=csv_coordinates by entering the list of coordinates, therefore I did so.

The data were then cleaned in OpenRefine and analysed through Pivot Tables, Pivot Charts and formulas (such as VLOOKUP) in Excel.